In [1]:
import numpy as np
import matplotlib.pyplot as plt  
import math   
import random   
import copy
import MEF

In [2]:
MN = np.array([[0,0,0],[8,0,0],[16,0,0],[8,4,0]]) #matriz de nodo
MC= np.array([[0,3],[0,1],[1,3],[2,3],[1,2]]) #matriz de conectividad
glxn= 2 #grados de libertad por nodo
Nn = MN.shape[0] #cantidad de nodos
Ne, Nnxe= MC.shape #cantidad de elementos y nodos por elementos

In [3]:
E= np.array([210,210,210,210,210])*1E9 
A = np.array([10,10,20,10,10])*1E-4 #sección
k= 200 #constante de los resortes

In [4]:
K_global= np.zeros([glxn*Nn,glxn*Nn])

In [5]:
def k_elem_barra(MC,MN,Ee,Ae,e):
    
    nodos= MC[e]
    cord_nodo=np.array([MN[nodos[1],0]-MN[nodos[0],0],MN[nodos[1],1]-MN[nodos[0],1]])
    long= np.linalg.norm(cord_nodo)
    phi= np.arctan2(cord_nodo[1],cord_nodo[0])
    ke= Ee*Ae/long
    c=np.cos(phi)
    s=np.sin(phi)
    K_elem= ke*np.array([[c**2,c*s,-c**2,-c*s],[c*s,s*s,-c*s,-s*s],
                         [-c*c,-c*s,c*c,c*s],[-c*s,-s*s,c*s,s*s]])
    K_elem[np.abs(K_elem/K_elem.max()) < 1e-15] = 0

    return K_elem

In [6]:
k_elem= {}

In [7]:
archivo= 'Matrices_elementales.txt'
with open(archivo,'w') as f:
    f.write('Matrices Elementales\n ===============')
archivo1= 'Matriz_global.txt'
with open(archivo1,'w') as f:
    f.write('Matriz Global\n ===============')

In [8]:
for e in range(Ne):
    k_elem[e]= k_elem_barra(MC,MN,E[e],A[e],e)
    fe = np.abs(k_elem[e].max()) #factor de escala
    with open(archivo,'a') as f:
        f.write(f'\nelemento {e}, fe ={fe:4e}\n')
        f.write(f'{k_elem[e]/fe}\n')
    for i in range(Nnxe):
        rangoi= np.linspace(i*glxn,(i+1)*glxn-1,glxn, dtype = "int")
        rangoni= np.linspace(MC[e,i]*glxn,(MC[e,i]+1)*glxn-1,glxn, dtype = "int")
        for j in range(Nnxe):
            rangoj= np.linspace(j*glxn,(j+1)*glxn-1,glxn, dtype = "int")
            rangonj= np.linspace(MC[e,j]*glxn,(MC[e,j]+1)*glxn-1,glxn, dtype = "int")
            K_global[np.ix_(rangoni,rangonj)] += k_elem[e][np.ix_(rangoi,rangoj)]

In [9]:
fe = np.abs(K_global.max())
with open(archivo1,'a') as f:
    f.write(f'\nMatriz Global, fe ={fe:4e}\n')
    f.write(f'{K_global/fe}\n') 

In [10]:
s= np.array([0,1,5]) # nodo con condiciones de vinculo
r = np.array([i for i in range(Nn*glxn) if i not in s ]) #complemento de s, incognitas

In [11]:
U_s = np.zeros_like(s)
F_r = np.zeros_like(r)
F_r[r==3]= -20E3

In [12]:
desp, fuerzas = MEF.solve1D(K_global,s,r,U_s,F_r)

In [13]:
despxy= desp.reshape(Nn,2)

In [14]:
despxy

array([[ 0.        ,  0.        ],
       [ 0.0007619 , -0.00384387],
       [ 0.00152381,  0.        ],
       [ 0.0007619 , -0.0036534 ]])

In [15]:
b= np.zeros([Nn,1])
despxyz= np.append(despxy, b, axis=1)

In [16]:
MN_final= despxyz + MN

In [17]:
Lf = []
Li=[]

In [18]:
sigma= []

In [19]:
for e in range(Ne):
    nodos= MC[e]
    cord_nodo_inicial=np.array([MN[nodos[1],0]-MN[nodos[0],0],MN[nodos[1],1]-MN[nodos[0],1]])
    cord_nodo_final=np.array([MN_final[nodos[1],0]-MN_final[nodos[0],0],
                              MN_final[nodos[1],1]-MN_final[nodos[0],1]])
    Lf.append(np.linalg.norm(cord_nodo_final))
    Li.append(np.linalg.norm(cord_nodo_inicial))
    sigma.append((((Lf[e]-Li[e])/Li[e])*E[e]))

In [20]:
sigma = np.array(sigma)
sigma/1E6 #pas las tensiones MPa

array([-22.34358817,  20.02423853,  10.        , -22.34358817,
        20.02423853])

In [21]:
Li 

[8.94427190999916, 8.0, 4.0, 8.94427190999916, 8.0]

<div style="background:#999900"> 
    Estoy revisando despues de terminar las clases, en el tren, de vacaciones. 
    El notebook anda pero le faltan los detalles. 
    Entregó muy tarde. considero resuelto pero incompleto
</div>